In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

# Data Cleaning and Preprocessing

Loading Data

In [2]:
DATA_DIR = '/Users/seancollins/Desktop/DATA 3000 Project/mens-march-mania-2022/MDataFiles_Stage2/'
for filename in os.listdir(DATA_DIR):
    print(filename)

MNCAATourneyDetailedResults.csv
MNCAATourneyCompactResults.csv
MSeasons.csv
MMasseyOrdinals_thruDay128.csv
MRegularSeasonDetailedResults.csv
MNCAATourneySlots.csv
MGameCities.csv
MConferenceTourneyGames.csv
Cities.csv
MRegularSeasonCompactResults.csv
MNCAATourneySeedRoundSlots.csv
MTeamConferences.csv
MSampleSubmissionStage2.csv
MTeamCoaches.csv
Conferences.csv
MTeams.csv
MNCAATourneySeeds.csv
MSecondaryTourneyTeams.csv
MTeamSpellings.csv
MSecondaryTourneyCompactResults.csv


In [3]:
reg_season = pd.read_csv(DATA_DIR + 'MRegularSeasonDetailedResults.csv')
tournament = pd.read_csv(DATA_DIR + 'MNCAATourneyDetailedResults.csv')
team_ids = pd.read_csv(DATA_DIR + 'MTeams.csv')
seeds = pd.read_csv(DATA_DIR + 'MNCAATourneySeeds.csv')
sample_submission = pd.read_csv(DATA_DIR + 'MSampleSubmissionStage2.csv')
conferences = pd.read_csv(DATA_DIR + 'Conferences.csv')

In [4]:
reg_season.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [5]:
#home and away
reg_season['WLoc'] = reg_season['WLoc'].apply(lambda x: 1 if x=='H' else 0)
reg_season['LLoc'] = 1 - reg_season['WLoc']

In [6]:
reg_season.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF',
       'LLoc'],
      dtype='object')

Get each row in the dataframe to be unique to one team in one season

In [7]:
winners = pd.DataFrame()
losers = pd.DataFrame()

cols = ['Season', 'TeamID', 'Score', 'OppScore', 'Loc',
       'NumOT', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM','FTA', 'OR', 'DR',
       'Ast', 'TO', 'Stl', 'Blk', 'PF', 'OppFGM', 'OppFGA', 'OppFGM3', 'OppFGA3',
       'OppFTM', 'OppFTA', 'OppOR', 'OppDR', 'OppAst', 'OppTO', 'OppStl',
       'OppBlk', 'OppPF', 'OppLoc']

winners[cols] = reg_season[['Season', 'WTeamID', 'WScore', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 'LLoc']]

winners['Wins'] = 1

winners['Losses'] = 0

losers[cols] = reg_season[['Season', 'LTeamID', 'LScore', 'WScore', 'LLoc',
       'NumOT', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR',
       'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3',
       'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'WLoc']]

losers['Wins'] = 0

losers['Losses'] = 1

winnersandlosers = pd.concat([winners, losers])

In [8]:
winnersandlosers

,Season,TeamID,Score,OppScore,Loc,NumOT,FGM,FGA,FGM3,FGA3,...,OppOR,OppDR,OppAst,OppTO,OppStl,OppBlk,OppPF,OppLoc,Wins,Losses
0,2003,1104,68,62,0,0,27,58,3,14,...,10,22,8,18,9,2,20,1,1,0
1,2003,1272,70,63,0,0,26,62,8,20,...,20,25,7,12,8,6,16,1,1,0
2,2003,1266,73,61,0,0,24,58,8,18,...,31,22,9,12,2,5,23,1,1,0
3,2003,1296,56,50,0,0,18,38,3,9,...,17,20,9,19,4,3,23,1,1,0
4,2003,1400,77,71,0,0,30,61,6,14,...,21,15,12,10,7,1,14,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102027,2022,1272,53,71,1,0,22,51,1,10,...,13,21,20,9,10,3,16,0,0,1
102028,2022,1345,66,75,1,0,25,59,5,20,...,10,20,15,6,13,3,17,0,0,1
102029,2022,1172,62,64,1,0,19,51,9,27,...,4,29,8,11,5,0,17,0,0,1
102030,2022,1401,50,65,1,0,16,51,4,19,...,12,29,19,13,8,4,14,0,0,1


In [9]:
combined = winnersandlosers.groupby(['Season', 'TeamID']).sum()

In [10]:
combined['NumGames'] = combined['Wins'] + combined['Losses']

In [11]:
combined

Score  OppScore  Loc  NumOT  FGM   FGA  FGM3  FGA3  FTM  FTA  \
Season TeamID                                                                 
2003   1102     1603      1596   15      0  536  1114   219   583  312  479   
       1103     2127      2110   14      8  733  1508   147   434  514  698   
       1104     1940      1820   16      1  673  1601   178   556  416  586   
       1105     1866      1993   12      4  634  1602   197   540  401  568   
       1106     1781      1785   14      1  656  1548   171   494  298  461   
...              ...       ...  ...    ...  ...   ...   ...   ...  ...  ...   
2022   1468     1971      2043   12      1  739  1601   229   670  264  352   
       1469     1838      2054   13      1  647  1578   174   589  370  555   
       1470     1768      1843   12      1  621  1498   145   469  381  505   
       1471     1846      1934   12      1  629  1438   237   659  351  470   
       1472     1962      2112   10      0  697  1604   275   800  293  380   

               ...  OppDR  OppAst  OppTO  OppStl  OppBlk  OppPF  OppLoc  Wins  \
Season TeamID  ...                                                              
2003   1102    ...    564     256    363     152      44    514      13    12   
       1103    ...    595     418    414     173      77    606      13    13   
       1104    ...    634     327    388     155      89    539      12    17   
       1105    ...    686     411    489     244     109    496      14     7   
       1106    ...    626     330    422     246      89    452      14    13   
...            ...    ...     ...    ...     ...     ...    ...     ...   ...   
2022   1468    ...    655     380    316     147      69    438      17    16   
       1469    ...    712     402    332     223     110    545      14     9   
       1470    ...    650     382    431     135     145    490      16    11   
       1471    ...    668     354    304     171      69    458      15    11   
       1472    ...    705     361    284      94      85    434      17     7   

               Losses  NumGames  
Season TeamID                    
2003   1102        16        28  
       1103        14        27  
       1104        11        28  
       1105        19        26  
       1106        15        28  
...               ...       ...  
2022   1468        13        29  
       1469        18        27  
       1470        17        28  
       1471        16        27  
       1472        20        27  

[6892 rows x 34 columns]